[Home](Home.ipynb)

# Algorithms in Cryptography

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/30269841575/in/album-72157660337424600/" title="DSCF9928"><img src="https://live.staticflickr.com/5691/30269841575_8bea763a54_w.jpg" width="300" height="400" alt="DSCF9928"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>


Building mathematical knowledge up to where RSA (named for inventors Ron Rivest, Adi Shamir and Leonard Adleman) makes sense, gives you many useful concepts and insights along the way.

For example, we should start with one of the oldest algorithms in history, called Euclid's Method.  Euclid probably got it from some earlier source.

Python will help us, by expressing these algorithms succinctly.  We'll be able to test them interactively.

However, before we get to [Euclid's Algorithm (EA)](https://en.wikipedia.org/wiki/Euclidean_algorithm), and then [Euclid's Extended Algorithm (EEA)](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm) we should review the basic concepts of prime versus composite positive integers.

### Primes vs Composites

Primes have no divisors.  Trying to divide them by another integer always leaves some remainder.

Composities are products of prime factors and comprise the rest of the positive integers.  

All integers are either composite or prime.  

Telling which is which is not always easy, because huge composites with only two prime factors may look just like primes, and finding these factors can be extremely difficult.  

In fact, RSA depends on factoring large composites being next to impossible, if they're large enough.

### Modulo Arithmetic in Python

When we talk about factoring numbers, that gets us thinking about remainders, like if there is one.  We say m divides n "evenly" not because m goes into n an even number of times (which may be true too) but because there's no remainder.  m divides n with nothing left over.  That makes m a factor of n.

In [12]:
12 % 3  # no remainder, 3 divides 12 evenly

0

In [14]:
12 % 7  # yes remainder, 7 is not a factor of 12

5

In [16]:
divmod(100, 12)

(8, 4)

12 goes into 100 eight times, leaving a remainder of 4.

In [22]:
def always_true(n, m):
    q, r = divmod(n, m)
    print(q, r)
    return n == q * m + r  # should always be True

In [23]:
always_true(28398, 747)  # try a bunch of examples

38 12


True

### Prime Numbers

In [29]:
import primes  # a package (has __init__.py)

In [30]:
dir(primes)    # not everything is exposed

['PrimeNumbers',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'all_factors',
 'eratosthenes',
 'factors',
 'isprime',
 'primes_gen',
 'primesplay']

In [4]:
primes.factors(100)

(1, 2, 2, 5, 5)

In [5]:
primes.all_factors(100)

[1, 2, 4, 5, 10, 20, 25, 50, 100]

In [10]:
p = primes.primes_gen.PrimeNumbers()
[next(p) for _ in range(20)]

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]

### Coprimes (Strangers)

Coprimes are not necessarily prime themselves.  Two numbers are coprime if they have no factors in common.  We also call them "strangers" in that case.

For example 10 and 7 are coprime, but not 27 and 15.  Those two (27 and 15) have the common factor 3.  

So that means:  if the greatest common divisor (GCD) of two positive integers is > 1 (greater than 1), then these two integers are *not* coprime.  Otherwise, if their GCD == 1, they are.

In [24]:
def euclid(a, b):
    while a % b:         # when remainder is 0, b is gcd
        b, a = a % b, b  # chopping down to 1
    return b

In [25]:
euclid(10, 5)

5

In [26]:
euclid(27, 15)

3

In [27]:
euclid(10, 7)

1

So 10 and 7 are strangers.

How does Euclid's Method get the job done?

If I'm looking for the greatest divisor of a and b, I should first see if a divides b or b divides a, with no remainder.  If $b > a$, then divmod(a, b) is (0, a).

In [32]:
divmod(4, 12)

(0, 4)

This means a and b will swap in the next line:

In [34]:
a=4
b=12

print(a, b)  # before

if a % b:
    b, a = a % b, b

print(a, b)  # after
    

4 12
12 4


Therefore, we don't really care if a > b or b < a at the start.

Going forward, whenever there's a remainder, the question becomes "what divides both this new remainder, and the smaller of the two numbers just compared?"  

In other words, the problem keeps transferring to finding a divisor that works for the smaller size, and the remainder upon dividing into the larger size.  The quantities get smaller and smaller.  

Once 1 is reached, as the smaller size b, we're done, as gcd = b = 1 always divides into an integer with no remainder.  Remember, if gcd(a, b) is 1, then a and b are coprime.

To be continued...

* Fermat's Primality Test
* Carmichael Numbers
* Totatives and Totient
* Euler's Theorem
* RSA Algorithm